### Genderizer
Classificateur de genre : Utilisé dans le contexte de l'analyse linguistique ou du traitement automatique des langues.
Utiliser Pytorch


Feuille de route:
- faire un état de l'art rapide sur le web
    - Nooj
    - HuggingFace
- choisir un dictionaire gratuit open source > Dictionnaire de l’Académie française de 1932-35 (8e édition) - GLP3
- créer une base de données de noms communs étiquetés par genre
- faire une analyse des données

In [19]:
import os
from bs4 import BeautifulSoup
import sqlite3
import pandas as pd
import urllib.request
import re
from IPython.display import display, HTML

In [11]:
# importer la source au format Mobipocket
url = "https://www.ebooksgratuits.com/newsendbook.php?id=1609&format=mp"
urllib.request.urlretrieve(url, "aca.prc")

print("Download complete.")

Download complete.


In [8]:
pip install mobi

Note: you may need to restart the kernel to use updated packages.


In [2]:
!mobiunpack -r aca.prc ./


KindleUnpack v0.82
   Based on initial mobipocket version Copyright © 2009 Charles M. Hannum <root@ihack.net>
   Extensive Extensions and Improvements Copyright © 2009-2014 
       by:  P. Durrant, K. Hendricks, S. Siebert, fandrieu, DiapDealer, nickredding, tkeo.
   This program is free software: you can redistribute it and/or modify
   it under the terms of the GNU General Public License as published by
   the Free Software Foundation, version 3.
Unpacking Book...
Completed


In [14]:
# prend environ 5 minutes

# Read the HTML content from the file
with open('./mobi7/book.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Define the regular expression patterns to match <idx:orth> and <idx:entry> tags
pattern_orth = re.compile(r'<idx:orth value="[^"]*">')
# pattern_entry = re.compile(r'<idx:entry>.*?</idx:entry>', re.DOTALL)
pattern_entry = re.compile(r'</idx:entry>.*?<mbp:pagebreak/>', re.DOTALL)

# Find all matches for <idx:orth> and <idx:entry> tags
orth_matches = pattern_orth.findall(html_content)
entry_matches = pattern_entry.findall(html_content)


In [15]:

print("Number of <idx:orth> tags:", len(orth_matches))
print("Number of <idx:entry> tags:", len(entry_matches))

Number of <idx:orth> tags: 31933
Number of <idx:entry> tags: 31933


In [16]:

# Print the last extracted tags
print("Extracted <idx:orth> tags:")
for tag in orth_matches[-4:]:
    print(tag)

print("\nExtracted <idx:entry> tags:")
for tag in entry_matches[-4:]:
    print(tag)

Extracted <idx:orth> tags:
<idx:orth value="ZOUAVE">
<idx:orth value="ZUT">
<idx:orth value="ZYGOMA">
<idx:orth value="ZYGOMATIQUE">

Extracted <idx:entry> tags:
</idx:entry>
<h2><font size="+2" color="BLUE"><b>ZOUAVE</b></font></h2><br/> <font color="green">n. m.</font><br/>Soldat d'un corps d'infanterie servant pour la plus grande partie en Afrique du Nord. <p> <i>Zouaves pontificaux,</i> Corps de volontaires constitué en 1860 par le général de Lamoricière pour la défense des États pontificaux. <mbp:pagebreak/>
</idx:entry>
<h2><font size="+2" color="BLUE"><b>ZUT</b></font></h2><br/> <font color="green">interj.</font><br/>Exclamation familière pour exprimer l'impatience, le dépit, pour envoyer les gens au diable et signifier qu'on ne veut plus rien entendre. <mbp:pagebreak/>
</idx:entry>
<h2><font size="+2" color="BLUE"><b>ZYGOMA</b></font></h2><br/> <font color="green">n. m.</font><br/><dom>T. d'Anatomie</dom><br/>Os de la pommette de la joue. <mbp:pagebreak/>
</idx:entry>
<h2><font

In [18]:
import os
import sqlite3

db_file = "acad_pocket.db"         # SQLite database file



# === STEP 2: INITIALIZE SQLITE DATABASE ===
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Create a table for words and definitions
cursor.execute("""
    CREATE TABLE IF NOT EXISTS dictionary (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        word TEXT,
        definition TEXT
    )
""")


# Insert words and definitions into the database
for word, definition in zip(orth_matches, entry_matches):

    word = re.search(r'value="([^"]*)"', word).group(1)
    definition = definition.replace('</idx:entry>', '').replace('<mbp:pagebreak/>', '')
    
    try:
        cursor.execute("INSERT INTO dictionary (word, definition) VALUES (?, ?)", (word, definition))
        
    except sqlite3.IntegrityError:
        pass  # Ignore duplicate words

# === STEP 4: FINALIZE ===
conn.commit()
conn.close()

print(f"Dictionary has been saved to {db_file}")

Dictionary has been saved to acad_pocket.db


In [20]:
import sqlite3
import pandas as pd
from IPython.display import display, HTML

# Connect to the SQLite database
conn = sqlite3.connect('acad_pocket.db')

# Write a query to fetch data
query = """
SELECT * FROM dictionary
WHERE word = 'TOUR'
"""

# Use pandas to read the data into a DataFrame
df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Check if the DataFrame is empty
if df.empty:
    print("No data found")
else:
    print(df)
    for ht in df.definition:
        # Display the HTML content
        display(HTML(ht))

      id  word                                         definition
0  30201  TOUR  \n<h2><font size="+2" color="BLUE"><b>TOUR</b>...
1  30202  TOUR  \n<h2><font size="+2" color="BLUE"><b>TOUR</b>...


---

In [22]:
def list_tags(html_content):
    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')
    list_balises = set()
    # Iterate over all tags in the HTML
    for tag in soup.find_all(True):
        list_balises.add(tag.name)

    list_balises = sorted(list_balises)
    return list_balises

In [ ]:
# Read the HTML content from the file - environ 5 minutes
with open('./mobi7/book.html', 'r', encoding='utf-8') as file:
    html_content = file.read()
print(list_tags(html_content))

['a', 'b', 'body', 'br', 'div', 'dom', 'fm', 'font', 'guide', 'h2', 'h3', 'head', 'html', 'i', 'idx:entry', 'idx:orth', 'img', 'mbp:frameset', 'mbp:pagebreak', 'meta', 'p', 'reference', 'renv', 'sup']


In [26]:
# Charger le contenu du fichier HTML
with open('mobi7/book.html', 'r', encoding='utf-8') as file:
    content = file.read()

# Parser le contenu HTML
soup = BeautifulSoup(content, 'html.parser')

# Trouver toutes les balises uniques
unique_tags = set(tag.name for tag in soup.find_all())

# Afficher les balises uniques
print(unique_tags)

{'idx:entry', 'font', 'div', 'h3', 'br', 'guide', 'i', 'idx:orth', 'renv', 'meta', 'reference', 'img', 'html', 'head', 'mbp:frameset', 'fm', 'sup', 'body', 'mbp:pagebreak', 'b', 'a', 'h2', 'p', 'dom'}


In [27]:


# Charger le contenu du fichier HTML
with open('mobi7/book.html', 'r', encoding='utf-8') as file:
    content = file.read()

# Parser le contenu HTML
soup = BeautifulSoup(content, 'html.parser')

# Trouver toutes les balises uniques avec leurs attributs
unique_tags_with_attrs = {}
for tag in soup.find_all():
    if tag.name not in unique_tags_with_attrs:
        unique_tags_with_attrs[tag.name] = set()
    for attr in tag.attrs:
        unique_tags_with_attrs[tag.name].add(attr)

# Stocker les balises uniques avec leurs attributs dans un fichier texte
with open('unique_tags.txt', 'w', encoding='utf-8') as file:
    for tag, attrs in unique_tags_with_attrs.items():
        file.write(f"{tag}: {', '.join(attrs)}\n")

In [28]:
from bs4 import BeautifulSoup

# Charger le contenu du fichier HTML
with open('mobi7/book.html', 'r', encoding='utf-8') as file:
    content = file.read()

# Parser le contenu HTML
soup = BeautifulSoup(content, 'html.parser')

# Trouver toutes les balises uniques avec leurs attributs
unique_tags_with_attrs = {}
for tag in soup.find_all():
    if tag.name not in unique_tags_with_attrs:
        unique_tags_with_attrs[tag.name] = set()
    for attr in tag.attrs:
        if '=' in str(tag):
            unique_tags_with_attrs[tag.name].add(attr)

# Stocker les balises uniques avec leurs attributs dans un fichier texte
with open('unique_tags.txt', 'w', encoding='utf-8') as file:
    for tag, attrs in unique_tags_with_attrs.items():
        file.write(f"{tag}: {', '.join(attrs)}\n")